In [1]:
# Import libraries
import requests
import pandas as pd
from bokeh.plotting import output_file, show
from bokeh.models import Select, Div, Button, ColumnDataSource, HTMLTemplateFormatter,TableColumn, DataTable
from bokeh.models.widgets import DateRangePicker, DataTable, TableColumn
from bokeh.layouts import column
# Import necessary modules
from ipywidgets import interact, widgets, VBox, HBox
from IPython.display import display, HTML
from bokeh.io import push_notebook, show, output_notebook
import pandas as pd
# Import the necessary modules
from ipywidgets import HTML
from IPython.display import display, HTML as IPHTML

In [2]:
# Import all APIs form config.py
# WeatherAPI key
weather_key = "11da115d9a8d4556bd993522240506"

In [3]:
def get_access_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": 'gAJOLRiyFCjYVuCEIspShbBroNVLSgpw',
        "client_secret": 'ZDHiCX0h0NYIBm0B'
    }
 
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        # Extract the access token from the response JSON
        access_token = response.json()['access_token']
        print("Access Token:", access_token)
    else:
        # Print error message if the request was not successful
        print("Failed to obtain access token:", response.status_code, response.text)
    return access_token

# Get access token
access_token = get_access_token()

Access Token: k1FXEI4d0bSZUo9dseFlEfKrYBp3


## Get flights according to city, airport and departure date

In [4]:
# List of cities
cities = ['Madrid', 'London', 'Paris', 'Barcelona', 'Athens']
# Function to get flight details
def get_flight_details(access_token, originLocationCode, destinationLocationCode, departureDate):
    url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
    headers = {
        "Authorization": f'Bearer {access_token}',
        "Content-Type": "application/json"
    }
    params = {
        "originLocationCode": originLocationCode,
        "destinationLocationCode": destinationLocationCode,
        "departureDate": departureDate,
        "adults": 1,
        "max": 3
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to get flight details:", response.status_code, response.text)
        return None

In [5]:

# Function to process flight data
def process_flight_data(flight_offers_response):
    if flight_offers_response:
        # Extract flight details and create DataFrame
        flight_data = []
        for flight in flight_offers_response['data']:
            itinerary = flight['itineraries'][0]
            segment = itinerary['segments'][0]
            departure_airport = segment['departure']['iataCode']
            arrival_airport = segment['arrival']['iataCode']
            departure_time = segment['departure']['at'][-8:-3]
            arrival_time = segment['arrival']['at'][-8:-3]

            # Convert string time to hours and minutes
            departure_hour, departure_minute = map(int, departure_time.split(':'))
            arrival_hour, arrival_minute = map(int, arrival_time.split(':'))

            # Calculate duration in minutes
            duration_minutes = (arrival_hour * 60 + arrival_minute) - (departure_hour * 60 + departure_minute)

            # Convert duration to hours and minutes
            duration_hours = duration_minutes // 60
            duration_minutes = duration_minutes % 60

            duration_str = f"{duration_hours:02d}:{duration_minutes:02d}"

            price = float(flight['price']['total'])
            currency = flight['price']['currency']
            price_with_currency = f"{price} {currency}"

            # Combine price with currency
            flight_data.append([departure_airport, arrival_airport, departure_time, arrival_time, duration_str, price_with_currency])

        # Creating DataFrame
        df = pd.DataFrame(flight_data, columns=['DepartureAirport', 'ArrivalAirport', 'DepartureTime', 'ArrivalTime', 'Duration(hours)', 'PriceWithCurrency'])
        return df
    else:
        print("No flight offers data available.")
        return None

In [6]:
# Function to get airport codes based on city
def get_airport_codes(city):
    api_url = f'https://api.api-ninjas.com/v1/airports?city={city}'
    headers = {'X-Api-Key': 'nrc3S/XZWbfao3LeySY9hg==I1u1i9JSOP6NSPFw'}
    response = requests.get(api_url, headers=headers)
    if response.status_code == requests.codes.ok:
        airport_data = response.json()
        airport_dict = {}
        for airport in airport_data:
            airport_code = airport.get('iata')
            if airport_code:
                airport_name = airport.get('name')
                airport_dict[airport_name] = airport_code
        return airport_dict
    else:
        print("Error:", response.status_code, response.text)
        return {}

In [7]:
# Function to update airport options based on city selection
def update_airport_options(change):
    city = change['new']
    airports = get_airport_codes(city)
    if airports:
        airport_select.options = airports
        airport_select.value = list(airports.values())[0]

In [8]:


# Function to update flight offers
def update_flight_offers(city, access_token, originLocationCode, destinationLocationCode, departure_date):
    print("City:", city)
    print("Departure Date:", departure_date)
    print("Origin Location Code:", originLocationCode)
    print("Destination Location Code:", destinationLocationCode)
    
    flight_offers_response = get_flight_details(access_token, originLocationCode, destinationLocationCode, departure_date)
    print(flight_offers_response)
    
    if flight_offers_response:
        flight_data_table = process_flight_data(flight_offers_response)
        if flight_data_table is not None:
            return flight_data_table
    else:
        print("No flight offers found.")
        return None

# STOP RUNNING THE CODE 

In [9]:
from datetime import datetime, timedelta

def fetch_10_day_forecast(city, api_key):
    base_url = 'http://api.weatherapi.com/v1'
    url = f"{base_url}/forecast.json?key={api_key}&q={city}&days=10"
    response = requests.get(url)
    data = response.json()
    
    # Check if the response contains the necessary forecast data
    if 'forecast' in data and 'forecastday' in data['forecast']:
        forecast_data = data['forecast']['forecastday']
        extracted_data = []
        
        for day in forecast_data:
            date = day['date']
            temp = day['day']['avgtemp_c']
            humidity = day['day']['avghumidity']
            cloudiness = day['day']['condition']['text']
            wind_speed = day['day']['maxwind_kph']
            
            extracted_data.append({
                'city': city,
                'date': date,
                'temp': temp,
                'humidity': humidity,
                'condition': cloudiness,
                'wind_speed': wind_speed
            })
        
        return pd.DataFrame(extracted_data)
    else:
        print(f"No forecast data available for {city}")
        return pd.DataFrame()



In [10]:
# Function to handle update button click
def on_update_button_clicked(button):
    city = city_select.value
    access_token = get_access_token()
    originLocationCode = 'MAD'
    destinationLocationCode = airport_select.value
    departure_date = departure_date_picker.value.strftime('%Y-%m-%d')
    
    # Clear previous outputs
    flight_details_output.clear_output()
    weather_output.clear_output()
    
    with flight_details_output:
        # Fetch and display flight details
        flight_data_table = update_flight_offers(city, access_token, originLocationCode, destinationLocationCode, departure_date)
        if flight_data_table is not None:
            display(flight_data_table)
        else:
            print("No flight offers found.")
    
    with weather_output:
        # Fetch and display weather forecast
        # Replace with your actual API key
        forecast_df = fetch_10_day_forecast(city, weather_key)
        
        if not forecast_df.empty:
            display(forecast_df)
        else:
            print("No forecast data available.")

# Initialize Bokeh for output in the notebook
output_notebook()

# List of cities
cities = ['Madrid', 'London', 'Paris', 'Barcelona']

# Create widgets
city_select = widgets.Dropdown(options=cities, description='Select City')
airport_select = widgets.Dropdown(description='Select Airport')
departure_date_picker = widgets.DatePicker(description='Select Date')
update_button = widgets.Button(description='Get Flights')
flight_details_output = widgets.Output()  # Output widget to display flight details table
weather_output = widgets.Output()  # Output widget to display weather forecast

# Attach the observer to update airport options when the city is selected
def update_airport_options(change):
    city = change.new
    airports = get_airport_codes(city)
    if airports:
        airport_select.options = airports
        airport_select.value = list(airports.values())[0]

city_select.observe(update_airport_options, names='value')

# Attach the event handler to the update button
update_button.on_click(on_update_button_clicked)

# Define a container for the widgets
inputs = VBox([city_select, airport_select, departure_date_picker, update_button])

# Display the widgets and outputs
display(inputs, flight_details_output, weather_output)

Loading BokehJS ...

Output()

Output()

# ENOUGH ALREADY

# Re doing it

Comment on what the function does and the input/output

In [26]:
#flight_offers_response = get_flight_details(access_token, originLocationCode, destinationLocationCode, departureDate)


In [28]:
#df = process_flight_data(flight_offers_response)